# Look further down for the sktime section
the first part matches the commented in the "ML_standard_shuffle.ipynb"

In [1]:
import pandas as pd
from pandas.plotting import scatter_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [2]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

In [3]:
from sklearn import model_selection
from sklearn.pipeline import Pipeline

from sktime.classification.compose import (
    ColumnEnsembleClassifier,
    TimeSeriesForestClassifier,
)
from sktime.classification.dictionary_based import BOSSEnsemble
from sktime.classification.shapelet_based import MrSEQLClassifier
from sktime.datasets import load_basic_motions
from sktime.transformations.panel.compose import ColumnConcatenator

In [4]:
file_path_screwdriver = r"C:\Users\Daniel\OneDrive\Bachelorarbeit\Daten\Datenaufnahme\Schraubendreher\1\2021-02-08Griff_Schraubendreher_ml EULERWINKEL.json"
file_path_fist = r"C:\Users\Daniel\OneDrive\Bachelorarbeit\Daten\Datenaufnahme\Faust\1\2021-02-08Faust_ml EULERWINKEL.json"
df_screwdriver = pd.read_json(file_path_screwdriver)
df_fist = pd.read_json(file_path_fist)

In [5]:
df_screwdriver.head()
#df.sample(10)

,phi,theta,psi,accX,accY,accZ,nameOfTask,userName,timeStamp,frameNumber,recordingNumber,typeOfRecording
0,"[-3.099991755198194, -0.34176092745658804, -0....","[-0.25186420991818304, 0.105667557468056, -1.0...","[0.986599007872573, -0.023337713198158003, -3....","[0.052691999999999996, -0.00015600000000000002...","[-0.037947, 0.011047999999999999, 0.048052, 0....","[0.057099, -0.0039059999999999997, 0.039547, -...",2021-02-08Griff_Schraubendreher_ml,-1,2021-02-08 10:26:04.840999842,43291,1,6
1,"[-3.09635428331108, -0.350613732759508, -0.030...","[-0.24841092096738102, 0.11023523884091001, -1...","[0.9868686556399041, -0.026869497884002003, -3...","[0.017114, -0.01899, 0.102838, 0.025011, 0.011...","[-0.000812, 0.021272, 0.023920999999999998, 0....","[-0.018637, 0.036375, 0.11451499999999999, -0....",2021-02-08Griff_Schraubendreher_ml,-1,2021-02-08 10:26:05.174000025,43301,1,6
2,"[-3.09635428331108, -0.350613732759508, -0.030...","[-0.24841092096738102, 0.11023523884091001, -1...","[0.9868686556399041, -0.026869497884002003, -3...","[0.017114, -0.023094, -0.031800999999999996, 0...","[-0.000812, -0.014856999999999999, -0.070271, ...","[-0.018637, -0.06293599999999999, -0.146164, 0...",2021-02-08Griff_Schraubendreher_ml,-1,2021-02-08 10:26:05.206999779,43302,1,6
3,"[-3.09635428331108, -0.351148677093969, -0.029...","[-0.24841092096738102, 0.11090753032435001, -1...","[0.9868686556399041, -0.027268226397941003, -3...","[0.046856999999999996, -0.040185, 0.025996, 0....","[-0.029146, -0.014199, -0.034952, -0.013939, 0...","[0.009911, 0.030994999999999998, 0.090491, 0.0...",2021-02-08Griff_Schraubendreher_ml,-1,2021-02-08 10:26:05.240999937,43303,1,6
4,"[-3.09635428331108, -0.351148677093969, -0.029...","[-0.24841092096738102, 0.11090753032435001, -1...","[0.9868686556399041, -0.027268226397941003, -3...","[0.007502, -0.005744, -0.029821999999999998, 0...","[-0.0037649999999999997, 0.021554, 0.002349, 0...","[0.011965, 0.013750000000000002, -0.0105919999...",2021-02-08Griff_Schraubendreher_ml,-1,2021-02-08 10:26:05.273999929,43304,1,6


In [6]:
delta_time = df_screwdriver["timeStamp"].iloc[266]-df_screwdriver["timeStamp"].iloc[265]
delta_time

Timedelta('-1 days +23:59:58.365999937')

In [7]:
df_screwdriver.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13176 entries, 0 to 13175
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   phi              13176 non-null  object        
 1   theta            13176 non-null  object        
 2   psi              13176 non-null  object        
 3   accX             13176 non-null  object        
 4   accY             13176 non-null  object        
 5   accZ             13176 non-null  object        
 6   nameOfTask       13176 non-null  object        
 7   userName         13176 non-null  int64         
 8   timeStamp        13176 non-null  datetime64[ns]
 9   frameNumber      13176 non-null  int64         
 10  recordingNumber  13176 non-null  int64         
 11  typeOfRecording  13176 non-null  int64         
dtypes: datetime64[ns](1), int64(4), object(7)
memory usage: 1.2+ MB


In [8]:
print(df_screwdriver.shape)
#print(df_screwdriver.describe())

(13176, 12)


In [9]:
df_screwdriver.drop(["userName","typeOfRecording","nameOfTask"],axis=1,inplace=True)
df_screwdriver.sort_values(["timeStamp"],ignore_index=True,inplace=True)
df_screwdriver.drop_duplicates(subset=["timeStamp"],keep="first",inplace=True,ignore_index=True)
df_screwdriver.head()

,phi,theta,psi,accX,accY,accZ,timeStamp,frameNumber,recordingNumber
0,"[-3.099991755198194, -0.34176092745658804, -0....","[-0.25186420991818304, 0.105667557468056, -1.0...","[0.986599007872573, -0.023337713198158003, -3....","[0.052691999999999996, -0.00015600000000000002...","[-0.037947, 0.011047999999999999, 0.048052, 0....","[0.057099, -0.0039059999999999997, 0.039547, -...",2021-02-08 10:26:04.840999842,43291,1
1,"[-3.099264195092009, -0.34283778181079605, -0....","[-0.25128315209456203, 0.10537330760081501, -1...","[0.9866276248304721, -0.022847743313952004, -3...","[0.023943, -0.045922, -0.058025999999999994, 0...","[-0.007743, -0.02128, -0.023585, -0.016905, -0...","[0.020853999999999998, -0.058086, -0.089409, 0...",2021-02-08 10:26:04.874000072,43292,1
2,"[-3.098991799395583, -0.343987736074917, -0.07...","[-0.2510143928558, 0.10580203429526001, -1.086...","[0.986707711360265, -0.023233381178098002, -3....","[0.044816999999999996, 0.004726999999999999, 0...","[0.023826999999999997, -0.001977, -0.013443, 0...","[0.030549, -0.029043, -0.0014529999999999999, ...",2021-02-08 10:26:04.907000065,43293,1
3,"[-3.098991799395583, -0.345417468001523, -0.06...","[-0.2510143928558, 0.10700270996869, -1.089496...","[0.986707711360265, -0.024144249907558, -3.030...","[0.044816999999999996, 0.017332999999999998, 0...","[0.023826999999999997, -0.00248, 0.005229, 0.0...","[0.030549, 0.015578999999999999, 0.089334, -0....",2021-02-08 10:26:04.940999985,43294,1
4,"[-3.098435052528825, -0.345417468001523, -0.06...","[-0.250482396131298, 0.10700270996869, -1.0894...","[0.986756380579293, -0.024144249907558, -3.030...","[0.024753999999999998, 0.017332999999999998, 0...","[-0.005931, -0.00248, 0.005229, 0.056212, 0.02...","[0.013267999999999999, 0.015578999999999999, 0...",2021-02-08 10:26:04.973999977,43295,1


In [10]:
df_fist.drop(["userName","typeOfRecording","nameOfTask"],axis=1,inplace=True)
df_fist.sort_values(["timeStamp"],ignore_index=True,inplace=True)
df_fist.drop_duplicates(subset=["timeStamp"],keep="first",inplace=True,ignore_index=True)
df_fist.head()

,phi,theta,psi,accX,accY,accZ,timeStamp,frameNumber,recordingNumber
0,"[-3.118076748816583, -0.333920426561088, -0.08...","[-0.26503149245521, 0.20645512801835, -1.16725...","[1.008453589306481, -0.08170777926853601, -3.0...","[0.023545, 0.016596, 0.068952, 0.0407849999999...","[-0.01305, 0.005895999999999999, 0.047337, 0.0...","[-0.020488, -0.018598999999999997, 0.026938999...",2021-02-08 10:31:40.942000151,53374,1
1,"[-3.118076748816583, -0.33608880099936805, -0....","[-0.26503149245521, 0.20753861472436602, -1.17...","[1.008453589306481, -0.08387754056954701, -3.0...","[-0.01855, -0.045995, -0.057689, -0.014369, -0...","[-0.001343, 0.050683, 0.080512, -0.01902000000...","[0.025713, -0.010338, 0.156693, -0.058301, 0.0...",2021-02-08 10:31:40.976000071,53375,1
2,"[-3.118076748816583, -0.339477831214323, -0.05...","[-0.26503149245521, 0.20871140553879003, -1.17...","[1.008453589306481, -0.086825528359548, -3.064...","[0.004666, -0.009868, -0.12695199999999998, 0....","[-0.015775, -0.0045839999999999995, -0.1189139...","[0.025092999999999997, -0.025686999999999998, ...",2021-02-08 10:31:41.008999825,53376,1
3,"[-3.118076748816583, -0.34093839439718004, -0....","[-0.26503149245521, 0.20955774880552702, -1.17...","[1.008453589306481, -0.08826124456152201, -3.0...","[0.004666, -0.035213999999999995, -0.078410000...","[-0.015775, -0.029311, -0.122308, 0.219354, -0...","[0.025092999999999997, -0.002561, -0.205281, 0...",2021-02-08 10:31:41.042000055,53377,1
4,"[-3.117801430967342, -0.342399496293784, -0.06...","[-0.265494503621495, 0.21288311175180602, -1.1...","[1.008747569826278, -0.090510446462067, -3.062...","[-0.005841, 0.012012, -0.006175, -0.0438700000...","[0.020560000000000002, 0.0018000000000000002, ...","[-0.007871, 0.017613, 0.074901, -0.046921, -0....",2021-02-08 10:31:41.075999975,53378,1


In [11]:
print("Number unique testing sequences SCREWDRIVER:\t {}".format(df_screwdriver["recordingNumber"].nunique()))
print("Number unique testing sequences FIST:\t\t {}".format(df_fist["recordingNumber"].nunique()))

Number unique testing sequences SCREWDRIVER:	 9
Number unique testing sequences FIST:		 10


In [12]:
n_sensors = len(df_screwdriver.loc[0]["phi"])
print(n_sensors)
df_screwdriver.loc[0]["phi"]

14


[-3.099991755198194,
 -0.34176092745658804,
 -0.072204900632534,
 -0.2748559103125,
 -0.15958225444509902,
 -0.099540365434666,
 0.0008232574914568344,
 0.080650762201317,
 -0.283347146365762,
 -0.070954656897366,
 0.080409062739526,
 0.198082098077199,
 0.230508702769312,
 0.225838294555115]

In [13]:
if (len(df_screwdriver.loc[0]["phi"])==len(df_fist.loc[0]["phi"])) : print("Same number of sensors used") 

Same number of sensors used


In [14]:
column_names_list = df_screwdriver.columns.tolist()
column_names_list.remove("timeStamp")
column_names_list.remove("frameNumber")
column_names_list.remove("recordingNumber")
column_names_list

['phi', 'theta', 'psi', 'accX', 'accY', 'accZ']

In [15]:
# List with 14 Dataframes, one for each sensor
all_sensors_list_screwdriver = []
temp_df = df_screwdriver.copy()
for sensor_index in range(n_sensors):
    for index, row in df_screwdriver.iterrows():
        for c_name in column_names_list:
            temp_df[c_name].iloc[index] = row[c_name][sensor_index]
    temp_df[column_names_list] = temp_df[column_names_list].astype(float)
    all_sensors_list_screwdriver.append(temp_df.copy())

C:\Users\Daniel\anaconda3\envs\gputest\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [16]:
# List with 14 Dataframes, one for each sensor
all_sensors_list_fist = []
temp_df = df_fist.copy()
for sensor_index in range(n_sensors):
    for index, row in df_fist.iterrows():
        for c_name in column_names_list:
            temp_df[c_name].iloc[index] = row[c_name][sensor_index]
    temp_df[column_names_list] = temp_df[column_names_list].astype(float)
    all_sensors_list_fist.append(temp_df.copy())

In [17]:
index_recNum_list_srewdriver = []
for recording in range(1,df_screwdriver["recordingNumber"].nunique()+1):
    index_recNum_list_srewdriver.append(df_screwdriver[df_screwdriver["recordingNumber"]==recording].first_valid_index())
index_recNum_list_srewdriver.append(df_screwdriver.last_valid_index()+1)
index_recNum_list_srewdriver

[0, 253, 488, 732, 944, 1204, 1426, 1699, 1924, 2136]

In [18]:
index_recNum_list_fist = []
for recording in range(1,df_fist["recordingNumber"].nunique()+1):
    index_recNum_list_fist.append(df_fist[df_fist["recordingNumber"]==recording].first_valid_index())
index_recNum_list_fist.append(df_fist.last_valid_index()+1)
index_recNum_list_fist

[0, 172, 378, 618, 873, 1107, 1317, 1593, 1879, 2167, 2447]

In [19]:
delta_time = []
for ind in range(1,len(index_recNum_list_fist)):
    t1 = df_fist["timeStamp"].iloc[index_recNum_list_fist[ind-1]]
    t2 = df_fist["timeStamp"].iloc[index_recNum_list_fist[ind]-1]
    delta_time.append((t2-t1))
arr = np.array(delta_time)
#print(arr)
print("durchschnittliche Zeit für eine SCHRAUBENDREHER Bewegung: \t"+str(arr.mean()))

durchschnittliche Zeit für eine SCHRAUBENDREHER Bewegung: 	0 days 00:00:08.122999930


In [20]:
delta_time = []
for ind in range(1,len(index_recNum_list_srewdriver)):
    t1 = df_screwdriver["timeStamp"].iloc[index_recNum_list_srewdriver[ind-1]]
    t2 = df_screwdriver["timeStamp"].iloc[index_recNum_list_srewdriver[ind]-1]
    delta_time.append((t2-t1))
arr = np.array(delta_time)
#print(arr)
print("durchschnittliche Zeit für eine FAUST Bewegung: \t"+str(arr.mean()))

durchschnittliche Zeit für eine FAUST Bewegung: 	0 days 00:00:07.877888970


In [21]:
# create entire Dataframe 
# label: 1=Scredriver;   2=Fist
data_srewdriver = df_screwdriver.copy()
data_srewdriver = data_srewdriver[["recordingNumber","timeStamp","frameNumber"]]
data_srewdriver

,recordingNumber,timeStamp,frameNumber
0,1,2021-02-08 10:26:04.840999842,43291
1,1,2021-02-08 10:26:04.874000072,43292
2,1,2021-02-08 10:26:04.907000065,43293
3,1,2021-02-08 10:26:04.940999985,43294
4,1,2021-02-08 10:26:04.973999977,43295
...,...,...,...
2131,9,2021-02-08 10:28:07.775000095,46979
2132,9,2021-02-08 10:28:07.806999922,46980
2133,9,2021-02-08 10:28:07.841000080,46981
2134,9,2021-02-08 10:28:07.875000000,46982


In [22]:
for ind_sensor in range(n_sensors):
    for ind_feat in range(6):
        col_name = column_names_list[ind_feat]+str(ind_sensor)
        data_srewdriver[col_name] = all_sensors_list_screwdriver[ind_sensor][column_names_list[ind_feat]]
data_srewdriver["label"] = 1
data_srewdriver.head()

,recordingNumber,timeStamp,frameNumber,phi0,theta0,psi0,accX0,accY0,accZ0,phi1,...,accX12,accY12,accZ12,phi13,theta13,psi13,accX13,accY13,accZ13,label
0,1,2021-02-08 10:26:04.840999842,43291,-3.099992,-0.251864,0.986599,0.052692,-0.037947,0.057099,-0.341761,...,-0.192187,-0.057048,0.335394,0.225838,-0.292762,0.669282,-0.224155,-0.062544,0.400648,1
1,1,2021-02-08 10:26:04.874000072,43292,-3.099264,-0.251283,0.986628,0.023943,-0.007743,0.020854,-0.342838,...,-0.080367,-0.088077,-0.171578,0.224768,-0.294493,0.670230,-0.040612,0.017532,-0.294340,1
2,1,2021-02-08 10:26:04.907000065,43293,-3.098992,-0.251014,0.986708,0.044817,0.023827,0.030549,-0.343988,...,0.079542,0.002126,-0.097973,0.224612,-0.294871,0.670486,0.106944,-0.051276,-0.009687,1
3,1,2021-02-08 10:26:04.940999985,43294,-3.098992,-0.251014,0.986708,0.044817,0.023827,0.030549,-0.345417,...,-0.051852,0.027972,-0.113384,0.225321,-0.297056,0.671869,-0.024738,-0.025348,-0.045350,1
4,1,2021-02-08 10:26:04.973999977,43295,-3.098435,-0.250482,0.986756,0.024754,-0.005931,0.013268,-0.345417,...,-0.040753,-0.019390,0.002805,0.226185,-0.297756,0.672643,-0.165805,-0.128674,-0.021833,1


In [23]:
data_fist = df_fist.copy()
data_fist = data_fist[["recordingNumber","timeStamp","frameNumber"]]
data_fist

,recordingNumber,timeStamp,frameNumber
0,1,2021-02-08 10:31:40.942000151,53374
1,1,2021-02-08 10:31:40.976000071,53375
2,1,2021-02-08 10:31:41.008999825,53376
3,1,2021-02-08 10:31:41.042000055,53377
4,1,2021-02-08 10:31:41.075999975,53378
...,...,...,...
2442,10,2021-02-08 10:33:58.108999968,57489
2443,10,2021-02-08 10:33:58.142999887,57490
2444,10,2021-02-08 10:33:58.176999807,57491
2445,10,2021-02-08 10:33:58.209000111,57492


all_sensors_list_fist

In [24]:
for ind_sensor in range(n_sensors):
    for ind_feat in range(6):
        col_name = column_names_list[ind_feat]+str(ind_sensor)
        data_fist[col_name] = all_sensors_list_fist[ind_sensor][column_names_list[ind_feat]]
data_fist["label"] = 2
data_fist.head()

,recordingNumber,timeStamp,frameNumber,phi0,theta0,psi0,accX0,accY0,accZ0,phi1,...,accX12,accY12,accZ12,phi13,theta13,psi13,accX13,accY13,accZ13,label
0,1,2021-02-08 10:31:40.942000151,53374,-3.118077,-0.265031,1.008454,0.023545,-0.013050,-0.020488,-0.333920,...,0.098876,0.499006,0.230281,0.155133,-0.239179,0.990093,0.259847,0.388906,0.373876,2
1,1,2021-02-08 10:31:40.976000071,53375,-3.118077,-0.265031,1.008454,-0.018550,-0.001343,0.025713,-0.336089,...,0.003207,0.123955,-0.175668,0.154752,-0.242360,0.998239,0.135430,0.264501,-0.252038,2
2,1,2021-02-08 10:31:41.008999825,53376,-3.118077,-0.265031,1.008454,0.004666,-0.015775,0.025093,-0.339478,...,-0.029397,-0.126940,0.210660,0.153998,-0.245201,1.004363,-0.128449,-0.103791,0.319710,2
3,1,2021-02-08 10:31:41.042000055,53377,-3.118077,-0.265031,1.008454,0.004666,-0.015775,0.025093,-0.340938,...,0.144977,-0.012840,0.086091,0.153208,-0.246982,1.009078,0.109099,0.022962,-0.131798,2
4,1,2021-02-08 10:31:41.075999975,53378,-3.117801,-0.265495,1.008748,-0.005841,0.020560,-0.007871,-0.342399,...,0.080459,-0.020516,-0.390120,0.151525,-0.248884,1.015170,0.054505,0.048795,-0.323801,2


In [25]:
data_fist.tail()

,recordingNumber,timeStamp,frameNumber,phi0,theta0,psi0,accX0,accY0,accZ0,phi1,...,accX12,accY12,accZ12,phi13,theta13,psi13,accX13,accY13,accZ13,label
2442,10,2021-02-08 10:33:58.108999968,57489,-3.101185,-0.259323,0.995941,-0.025081,-0.016308,-0.018873,-0.371827,...,-0.093461,-0.024301,0.113809,0.188865,-0.240217,0.950502,-0.041030,-0.013941,0.176515,2
2443,10,2021-02-08 10:33:58.142999887,57490,-3.101185,-0.259323,0.995941,0.007077,-0.048536,0.017174,-0.371407,...,-0.032861,-0.036520,-0.231586,0.188396,-0.240344,0.948447,-0.009833,0.002942,-0.117649,2
2444,10,2021-02-08 10:33:58.176999807,57491,-3.101185,-0.259323,0.995941,0.007077,-0.048536,0.017174,-0.371148,...,0.083403,-0.104163,0.091948,0.187876,-0.240420,0.947173,0.068939,-0.039344,-0.001130,2
2445,10,2021-02-08 10:33:58.209000111,57492,-3.101185,-0.259323,0.995941,-0.004127,-0.001223,0.027562,-0.371148,...,-0.002063,-0.048087,-0.077849,0.187152,-0.240386,0.945828,-0.051900,-0.089484,-0.055887,2
2446,10,2021-02-08 10:33:58.243000031,57493,-3.101185,-0.259323,0.995941,-0.000833,0.006183,-0.002618,-0.371148,...,-0.009213,0.078099,0.065679,0.186488,-0.240263,0.944539,0.051060,0.112721,0.046783,2


In [26]:
#data_fist.info()
data_fist.shape
#data_fist.describe()

(2447, 88)

In [27]:
#data_srewdriver.info()
data_srewdriver.shape
#data_srewdriver.describe()

(2136, 88)

In [28]:
t1 = data_srewdriver.copy()
t2 = data_fist.copy()
result = pd.concat([t1,t2],ignore_index=True)
result.shape

(4583, 88)

In [29]:
index_recNum_list_fist

[0, 172, 378, 618, 873, 1107, 1317, 1593, 1879, 2167, 2447]

In [30]:
index_recNum_list_fist = [x+index_recNum_list_srewdriver[-1]  for x in index_recNum_list_fist]
index_recNum_list_fist

[2136, 2308, 2514, 2754, 3009, 3243, 3453, 3729, 4015, 4303, 4583]

In [31]:
pure_data = result[result.columns[(3+6*7):]]
pure_data

,phi7,theta7,psi7,accX7,accY7,accZ7,phi8,theta8,psi8,accX8,...,accX12,accY12,accZ12,phi13,theta13,psi13,accX13,accY13,accZ13,label
0,0.080651,0.148847,-0.959140,-0.049675,0.134449,0.103026,-0.283347,-0.518190,-0.919459,-0.056956,...,-0.192187,-0.057048,0.335394,0.225838,-0.292762,0.669282,-0.224155,-0.062544,0.400648,1
1,0.080651,0.148847,-0.959140,-0.049675,0.134449,0.103026,-0.286494,-0.523345,-0.917932,-0.138175,...,-0.080367,-0.088077,-0.171578,0.224768,-0.294493,0.670230,-0.040612,0.017532,-0.294340,1
2,0.083520,0.145020,-0.958043,0.155792,-0.073801,-0.054326,-0.288562,-0.524751,-0.916598,-0.120184,...,0.079542,0.002126,-0.097973,0.224612,-0.294871,0.670486,0.106944,-0.051276,-0.009687,1
3,0.084042,0.144297,-0.957704,0.083194,-0.060714,-0.035120,-0.289412,-0.525137,-0.915855,-0.085881,...,-0.051852,0.027972,-0.113384,0.225321,-0.297056,0.671869,-0.024738,-0.025348,-0.045350,1
4,0.084970,0.142817,-0.957260,0.080727,-0.063714,-0.082944,-0.291794,-0.527175,-0.915196,-0.099641,...,-0.040753,-0.019390,0.002805,0.226185,-0.297756,0.672643,-0.165805,-0.128674,-0.021833,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4578,-0.000788,0.131263,-0.888502,-0.076930,-0.029085,0.101990,-0.383999,-0.487630,-0.916883,-0.021625,...,-0.093461,-0.024301,0.113809,0.188865,-0.240217,0.950502,-0.041030,-0.013941,0.176515,2
4579,0.000603,0.129653,-0.889342,-0.013478,0.004246,-0.029982,-0.383999,-0.487630,-0.916883,-0.021625,...,-0.032861,-0.036520,-0.231586,0.188396,-0.240344,0.948447,-0.009833,0.002942,-0.117649,2
4580,0.001227,0.128868,-0.889633,0.012872,-0.102590,-0.082346,-0.383633,-0.488823,-0.918637,-0.029767,...,0.083403,-0.104163,0.091948,0.187876,-0.240420,0.947173,0.068939,-0.039344,-0.001130,2
4581,0.001227,0.128868,-0.889633,0.012872,-0.102590,-0.082346,-0.383148,-0.490075,-0.920805,0.028057,...,-0.002063,-0.048087,-0.077849,0.187152,-0.240386,0.945828,-0.051900,-0.089484,-0.055887,2


# Data preparation 

## structure the data, so there is:
   ## one row for every test-sequence (9 for the screwdriver and 10 for the fist) 
   ## one column for every sensor-dimension (6 dimensions for each of the 14 sensors (only right hand movement -> 7 sensors)) 

In [32]:
cols = []
cols.extend(list(pure_data.columns[:6]))
cols.append("label")
cols

['phi7', 'theta7', 'psi7', 'accX7', 'accY7', 'accZ7', 'label']

In [33]:
list_df_testsequences_screwdriver = []
for ind in range(len(index_recNum_list_srewdriver)-1):
    temp_df = pure_data.iloc[index_recNum_list_srewdriver[ind]:index_recNum_list_srewdriver[ind+1]]
    list_df_testsequences_screwdriver.append(temp_df)
list_df_testsequences_screwdriver

[         phi7    theta7      psi7     accX7     accY7     accZ7      phi8  \
 0    0.080651  0.148847 -0.959140 -0.049675  0.134449  0.103026 -0.283347   
 1    0.080651  0.148847 -0.959140 -0.049675  0.134449  0.103026 -0.286494   
 2    0.083520  0.145020 -0.958043  0.155792 -0.073801 -0.054326 -0.288562   
 3    0.084042  0.144297 -0.957704  0.083194 -0.060714 -0.035120 -0.289412   
 4    0.084970  0.142817 -0.957260  0.080727 -0.063714 -0.082944 -0.291794   
 ..        ...       ...       ...       ...       ...       ...       ...   
 248  0.016378  0.213696 -0.962492  0.017214 -0.006230  0.068891 -0.366651   
 249  0.016378  0.213696 -0.962492  0.017214 -0.006230  0.068891 -0.366651   
 250  0.016202  0.213472 -0.962622 -0.011703 -0.026198  0.068336 -0.366841   
 251  0.016150  0.213309 -0.962629  0.039405 -0.023967  0.047134 -0.367714   
 252  0.016150  0.213309 -0.962629  0.039405 -0.023967  0.047134 -0.368434   
 
        theta8      psi8     accX8  ...    accX12    accY12   

In [34]:
list_df_testsequences_fist = []
for ind in range(len(index_recNum_list_fist)-1):
    temp_df = pure_data.iloc[index_recNum_list_fist[ind]:index_recNum_list_fist[ind+1]]
    list_df_testsequences_fist.append(temp_df)
#list_df_testsequences_fist

In [35]:
list_df_testsequences_screwdriver[0]["phi7"]

0      0.080651
1      0.080651
2      0.083520
3      0.084042
4      0.084970
         ...   
248    0.016378
249    0.016378
250    0.016202
251    0.016150
252    0.016150
Name: phi7, Length: 253, dtype: float64

In [36]:
arr_screwdriver = np.empty((9,43),dtype=object)
for ind_df in range(9):
    df_temp = list_df_testsequences_screwdriver[ind_df]
    for ind_col,col in enumerate(df_temp.columns,start=0):
        arr_screwdriver[ind_df,ind_col] = df_temp[col].values

In [38]:
arr_fist = np.empty((10,43),dtype=object)
for ind_df in range(10):
    df_temp = list_df_testsequences_fist[ind_df]
    for ind_col,col in enumerate(df_temp.columns,start=0):
        arr_fist[ind_df,ind_col] = df_temp[col].values

In [39]:
X_screwdriver = pd.DataFrame(arr_screwdriver[:,:-1])
y_screwdriver = pd.Series(np.ones(9))
# y contains the labels

In [40]:
X_fist = pd.DataFrame(arr_fist[:,:-1])
y_fist = pd.Series(np.full((10),2))

In [41]:
X = pd.concat((X_screwdriver,X_fist),ignore_index=True)

In [42]:
y = pd.concat((y_screwdriver,y_fist),ignore_index=True)

In [43]:
X

,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,41
0,"[0.080650762201317, 0.080650762201317, 0.08352...","[0.14884709394158802, 0.14884709394158802, 0.1...","[-0.9591402148408511, -0.9591402148408511, -0....","[-0.049675, -0.049675, 0.15579199999999999, 0....","[0.13444899999999999, 0.13444899999999999, -0....","[0.10302599999999999, 0.10302599999999999, -0....","[-0.283347146365762, -0.286493762435496, -0.28...","[-0.518189777987707, -0.5233446281637341, -0.5...","[-0.9194593668610541, -0.9179317838612471, -0....","[-0.056956, -0.138175, -0.120184, -0.085881, -...",...,"[0.653562569015073, 0.6546653731893151, 0.6555...","[-0.192187, -0.080367, 0.079542, -0.0518519999...","[-0.057047999999999995, -0.088077, 0.002126, 0...","[0.33539399999999997, -0.17157799999999998, -0...","[0.225838294555115, 0.224768064890726, 0.22461...","[-0.292762278942131, -0.29449338018408905, -0....","[0.6692819441285931, 0.670229732250855, 0.6704...","[-0.224155, -0.040611999999999995, 0.106944, -...","[-0.062544, 0.017532, -0.051275999999999995, -...","[0.400648, -0.29434000000000005, -0.009687, -0..."
1,"[0.019562524206611, 0.019562524206611, 0.01857...","[0.21648722383267502, 0.21648722383267502, 0.2...","[-0.9312109548694241, -0.9312109548694241, -0....","[0.021570000000000002, 0.021570000000000002, 0...","[0.075442, 0.075442, -0.016124, 0.047042, 0.04...","[0.019731, 0.019731, -0.07994799999999999, 0.0...","[-0.40842112138596803, -0.41336384490027306, -...","[-0.532898537205717, -0.533941244960449, -0.53...","[-0.832770126881037, -0.8311457766638811, -0.8...","[0.073559, 0.052905999999999995, 0.056219, -0....",...,"[0.734647685985483, 0.7346579668162441, 0.7348...","[0.037231, 0.046627999999999996, -0.074567, 0....","[0.046225999999999996, -0.006880000000000001, ...","[0.005680999999999999, -0.010635, -0.094458, 0...","[0.175866707238229, 0.17607507064421202, 0.176...","[-0.34624478430001504, -0.34572803471261704, -...","[0.6072413427715191, 0.606820258693065, 0.6068...","[0.065358, 0.076127, -0.016704, 0.074395, -0.0...","[0.021376, -0.039839, -0.028973, -0.030052, 0....","[0.070649, -0.07596499999999999, 0.003276, 0.1..."
2,"[-0.06566833649362301, -0.06566833649362301, -...","[0.23495997823754702, 0.23495997823754702, 0.2...","[-0.9748712887374841, -0.9748712887374841, -0....","[0.009471, -0.003251, 0.045861, 0.011023, 0.01...","[0.131871, 0.067296, -0.088458, -0.111036, -0....","[0.019318, 0.011397, -0.022896, -0.060141, -0....","[-0.5550295612389631, -0.5550295612389631, -0....","[-0.518840315630419, -0.518840315630419, -0.51...","[-0.763432005149536, -0.763432005149536, -0.76...","[0.038688, -0.055062, 0.016534, 0.063130999999...",...,"[0.6561374556826971, 0.6561374556826971, 0.656...","[0.070293, 0.08946899999999999, -0.016198, -0....","[0.144531, 0.155746, -0.063196, -0.160071, 0.0...","[0.17496299999999998, -0.026982, -0.159491, 0....","[0.15772848535894002, 0.15772848535894002, 0.1...","[-0.26977689601014604, -0.26977689601014604, -...","[0.5473196749443621, 0.5473196749443621, 0.547...","[0.142119, 0.13691799999999998, -0.088753, -0....","[0.129326, 0.110853, -0.05892000000000001, -0....","[0.123848, -0.001205, -0.157333, 0.132814, -0...."
3,"[-0.014659126188947002, -0.014659126188947002,...","[0.13950424247630502, 0.13950424247630502, 0.1...","[-0.9655338866420561, -0.9655338866420561, -0....","[-0.000777, -0.000777, 0.100035, 0.021821, 0.0...","[0.071682, 0.071682, -0.028083999999999998, 0....","[0.08270000000000001, 0.08270000000000001, -0....","[-0.571226858837385, -0.571226858837385, -0.57...","[-0.6410096358335361, -0.6410096358335361, -0....","[-0.780697285173336, -0.780697285173336, -0.78...","[0.021925, 0.0063549999999999995, 0.041351, 0....",...,"[0.6760564573687241, 0.6760564573687241, 0.676...","[-0.051828, 0.10202299999999999, 0.043467, 0.0...","[0.027004, 0.051144999999999996, -0.0904239999...","[-0.074339, 0.043920999999999995, -0.128332, 0...","[0.28985105043882, 0.289466033810722, 0.289110...","[-0.3258557654775100

In [44]:
length_list = []
for index,row in X.iterrows():
    length_list.append(len(row.loc[0]))
print(np.array(length_list).min())

172


In [45]:
# sktime needs each testing-sequence to have the same length (= same time length)
temp_list = []
cols = list(X.columns)
for index,row in X.iterrows():
    for ind_col, col in enumerate(cols):
        X.loc[index,ind_col] = np.array(list(row[col])[:170])
X

,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,41
0,"[0.080650762201317, 0.080650762201317, 0.08352...","[0.14884709394158802, 0.14884709394158802, 0.1...","[-0.9591402148408511, -0.9591402148408511, -0....","[-0.049675, -0.049675, 0.15579199999999999, 0....","[0.13444899999999999, 0.13444899999999999, -0....","[0.10302599999999999, 0.10302599999999999, -0....","[-0.283347146365762, -0.286493762435496, -0.28...","[-0.518189777987707, -0.5233446281637341, -0.5...","[-0.9194593668610541, -0.9179317838612471, -0....","[-0.056956, -0.138175, -0.120184, -0.085881, -...",...,"[0.653562569015073, 0.6546653731893151, 0.6555...","[-0.192187, -0.080367, 0.079542, -0.0518519999...","[-0.057047999999999995, -0.088077, 0.002126, 0...","[0.33539399999999997, -0.17157799999999998, -0...","[0.225838294555115, 0.224768064890726, 0.22461...","[-0.292762278942131, -0.29449338018408905, -0....","[0.6692819441285931, 0.670229732250855, 0.6704...","[-0.224155, -0.040611999999999995, 0.106944, -...","[-0.062544, 0.017532, -0.051275999999999995, -...","[0.400648, -0.29434000000000005, -0.009687, -0..."
1,"[0.019562524206611, 0.019562524206611, 0.01857...","[0.21648722383267502, 0.21648722383267502, 0.2...","[-0.9312109548694241, -0.9312109548694241, -0....","[0.021570000000000002, 0.021570000000000002, 0...","[0.075442, 0.075442, -0.016124, 0.047042, 0.04...","[0.019731, 0.019731, -0.07994799999999999, 0.0...","[-0.40842112138596803, -0.41336384490027306, -...","[-0.532898537205717, -0.533941244960449, -0.53...","[-0.832770126881037, -0.8311457766638811, -0.8...","[0.073559, 0.052905999999999995, 0.056219, -0....",...,"[0.734647685985483, 0.7346579668162441, 0.7348...","[0.037231, 0.046627999999999996, -0.074567, 0....","[0.046225999999999996, -0.006880000000000001, ...","[0.005680999999999999, -0.010635, -0.094458, 0...","[0.175866707238229, 0.17607507064421202, 0.176...","[-0.34624478430001504, -0.34572803471261704, -...","[0.6072413427715191, 0.606820258693065, 0.6068...","[0.065358, 0.076127, -0.016704, 0.074395, -0.0...","[0.021376, -0.039839, -0.028973, -0.030052, 0....","[0.070649, -0.07596499999999999, 0.003276, 0.1..."
2,"[-0.06566833649362301, -0.06566833649362301, -...","[0.23495997823754702, 0.23495997823754702, 0.2...","[-0.9748712887374841, -0.9748712887374841, -0....","[0.009471, -0.003251, 0.045861, 0.011023, 0.01...","[0.131871, 0.067296, -0.088458, -0.111036, -0....","[0.019318, 0.011397, -0.022896, -0.060141, -0....","[-0.5550295612389631, -0.5550295612389631, -0....","[-0.518840315630419, -0.518840315630419, -0.51...","[-0.763432005149536, -0.763432005149536, -0.76...","[0.038688, -0.055062, 0.016534, 0.063130999999...",...,"[0.6561374556826971, 0.6561374556826971, 0.656...","[0.070293, 0.08946899999999999, -0.016198, -0....","[0.144531, 0.155746, -0.063196, -0.160071, 0.0...","[0.17496299999999998, -0.026982, -0.159491, 0....","[0.15772848535894002, 0.15772848535894002, 0.1...","[-0.26977689601014604, -0.26977689601014604, -...","[0.5473196749443621, 0.5473196749443621, 0.547...","[0.142119, 0.13691799999999998, -0.088753, -0....","[0.129326, 0.110853, -0.05892000000000001, -0....","[0.123848, -0.001205, -0.157333, 0.132814, -0...."
3,"[-0.014659126188947002, -0.014659126188947002,...","[0.13950424247630502, 0.13950424247630502, 0.1...","[-0.9655338866420561, -0.9655338866420561, -0....","[-0.000777, -0.000777, 0.100035, 0.021821, 0.0...","[0.071682, 0.071682, -0.028083999999999998, 0....","[0.08270000000000001, 0.08270000000000001, -0....","[-0.571226858837385, -0.571226858837385, -0.57...","[-0.6410096358335361, -0.6410096358335361, -0....","[-0.780697285173336, -0.780697285173336, -0.78...","[0.021925, 0.0063549999999999995, 0.041351, 0....",...,"[0.6760564573687241, 0.6760564573687241, 0.676...","[-0.051828, 0.10202299999999999, 0.043467, 0.0...","[0.027004, 0.051144999999999996, -0.0904239999...","[-0.074339, 0.043920999999999995, -0.128332, 0...","[0.28985105043882, 0.289466033810722, 0.289110...","[-0.3258557654775100

In [56]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y,test_size=0.2, random_state=42)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(15, 42) (15,) (4, 42) (4,)


In [57]:
clf = MrSEQLClassifier(seql_mode="fs",symrep=["sax","sfa"])
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

1.0

In [58]:
prediction = clf.predict(X_test)
prediction

array([1., 1., 2., 1.])

In [59]:
steps = [
    ("concatenate", ColumnConcatenator()),
    ("classify", TimeSeriesForestClassifier(n_estimators=100)),
]
clf = Pipeline(steps)
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

1.0

In [60]:
clf = ColumnEnsembleClassifier(
    estimators=[
        ("TSF0", TimeSeriesForestClassifier(n_estimators=100), [0]),
        ("BOSSEnsemble3", BOSSEnsemble(max_ensemble_size=5), [3]),
    ]
)
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

1.0